<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Lesson 5: Uncertainty</h1>


<a name='section_5_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L5.0 Overview</h2>


<h3>Navigation</h3>

<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_5_1">L5.1 What Do We Call Uncertainty?</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_5_1">L5.1 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_5_2">L5.2 Extracting Uncertainty For Linear Fit</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_5_2">L5.2 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_5_3">L5.3 Computing Uncertainty</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_5_3">L5.3 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_5_4">L5.4 Introduction to Likelihood</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_5_4">L5.4 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_5_5">L5.5 An Example: Auger Data (Part 1)</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_5_5">L5.5 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_5_6">L5.6 An Example: Auger Data (Part 2)</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_5_6">L5.6 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_5_7">L5.7 Log-Likelihood and Chi-square</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_5_7">L5.7 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_5_8">L5.8 Minimizing</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_5_8">L5.8 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_5_9">L5.9 Comparison Using lmfit</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_5_9">L5.9 Exercises</a></td>
    </tr>
</table>



<h3>Importing Data (Colab Only)</h3>

If you are in a Google Colab environment, run the cell below to import the data for this notebook. Otherwise, if you have downloaded the course repository, you do not have to run the cell below.

See the source and attribution information below:

>data: data/L05/sn_z_mu_dmu_plow_union2.1.txt <br>
>source: http://supernova.lbl.gov/Union/ <br>
>attribution: The Supernova Cosmology Project, arXiv:1105.3470v1 <br>
>license type: https://arxiv.org/licenses/nonexclusive-distrib/1.0/license.html 

>data: data/L05/events_a4_1space.dat, data/L05/events_a8_1space.dat <br>
>source: https://www.auger.org/index.php/science/data <br>
>source : https://arxiv.org/abs/1709.07321 <br>
>attribution: Pierre Auger Collaboration, arXiv:1709.07321v1 <br>
>license type: https://creativecommons.org/licenses/by-sa/4.0/ 

In [ ]:
#>>>RUN: L5.0-runcell02

#importing data from git repository

!git init
!git remote add -f origin https://github.com/mitx-8s50/nb_LEARNER/
!git config core.sparseCheckout true
!echo 'data/L05' >> .git/info/sparse-checkout
!git pull origin main

In [ ]:
#>>>RUN: L5.0-runcell01

!pip install lmfit
!pip install astropy

In [ ]:
#>>>RUN: L5.0-runcell02

import numpy as np               #https://numpy.org/doc/stable/
import matplotlib.pyplot as plt  #https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html
import csv                       #https://docs.python.org/3/library/csv.html 
import math                      #https://docs.python.org/3/library/math.html
from scipy import stats          #https://docs.scipy.org/doc/scipy/reference/stats.html

In [ ]:
#>>>RUN: L5.0-runcell02

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title


<a name='section_5_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L5.1 What Do We Call Uncertainty?</h2>  

| [Top](#section_5_0) | [Previous Section](#section_5_0) | [Exercises](#exercises_5_1) | [Next Section](#section_5_2) |


In [ ]:
#>>>RUN: L5.1-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L05/slides_L05_01.html', width=970, height=550)

In [ ]:
#>>>RUN: L5.1-runcell01

import numpy as np
import csv
import math

#Let's try to understand how good the fits we made in last Lesson are, let's load the supernova data again
label='data/L05/sn_z_mu_dmu_plow_union2.1.txt'


def distanceconv(iMu):
    power=iMu/5+1
    return 10**power

def distanceconverr(iMu,iMuErr):
    power=iMu/5+1
    const=math.log(10)/5.
    return const*(10**power)*iMuErr

#Now let's zoom in on the small redshift data
def load(iLabel,iZMax):
    redshift=np.array([])
    distance=np.array([])
    distance_err=np.array([])
    with open(label,'r') as csvfile:
        plots = csv.reader(csvfile, delimiter='\t')
        for row in plots:
            if float(row[1]) > iZMax:
                continue
            redshift = np.append(redshift,float(row[1]))
            distance = np.append(distance,distanceconv(float(row[2])))
            distance_err = np.append(distance_err,distanceconverr(float(row[2]),float(row[3])))
    return redshift,distance,distance_err

redshift=np.array([])
distance=np.array([])
distance_err=np.array([])
redshift,distance,distance_err = load(label,0.1)

In [ ]:
#>>>RUN: L5.1-runcell02

import matplotlib.pyplot as plt


#Now let's run the regression again
def variance(isamples):
    mean=isamples.mean()
    n=len(isamples)
    tot=0
    for pVal in isamples:
        tot+=(pVal-mean)**2
    return tot/n

def covariance(ixs,iys):
    meanx=ixs.mean()
    meany=iys.mean()
    n=len(ixs)
    tot=0
    for i0 in range(len(ixs)):
        tot+=(ixs[i0]-meanx)*(iys[i0]-meany)
    return tot/n

def linear(ix,ia,ib):
    return ia*ix+ib

def regress(redshift,distance):
    #Let's regress
    var=variance(redshift)
    cov=covariance(redshift,distance)
    A=cov/var
    b=distance.mean()-A*redshift.mean()
    #Done!
    return A,b

def plotAll(redshift,distance,distance_err,A,b):
    #now let's plot it
    xmax=np.max(redshift)
    xvals = np.linspace(0,xmax,100)
    yvals = []
    for pX in xvals:
        yvals.append(linear(pX,A,b))

    #Plot the line
    plt.plot(xvals,yvals)
    plt.errorbar(redshift,distance,yerr=distance_err,marker='.',linestyle = 'None', color = 'black')
    plt.xlabel('redshift(z)', fontsize=15) #Label x
    plt.ylabel('distances(parsec)', fontsize=15)#Label y
    plt.show()
    #Print it out
    print("Hubbles Constant:",1e6*3e5/A,"intercept",b)#Note 1e6 is from pc to Mpc and 3e5 is c in km/s

A,b=regress(redshift,distance)
plotAll(redshift,distance,distance_err,A,b)

In [ ]:
#>>>RUN: L5.1-runcell03

from scipy import stats

def residual(func,args,redshift,distance,distance_err=[]):
    residuals=np.array([])
    for i0 in range(len(redshift)):
        pResid=func(redshift[i0],args[0],args[1])-distance[i0]
        if len(distance_err) > 0:      
            pResid=pResid/distance_err[i0]
        residuals = np.append(residuals,pResid)
    return residuals

#This time we are going to look at a histogram of the residuals
def plotHist(residuals):
    y0, bin_edges = np.histogram(residuals, bins=30)
    bin_centers = 0.5*(bin_edges[1:] + bin_edges[:-1])
    norm0=len(residuals)*(bin_edges[-1]-bin_edges[0])/30.
    plt.errorbar(bin_centers,y0/norm0,yerr=y0**0.5/norm0,marker='.',drawstyle = 'steps-mid')
    
    #for good measure, let's compare this to a gaussian distribution
    k=np.linspace(bin_edges[0],bin_edges[-1],100)
    normal=stats.norm.pdf(k,0,residuals.std())
    plt.plot(k,normal,'o-')
    plt.xlabel("y$_{residual}$")
    plt.ylabel("probability")
    plt.show()
    
residuals=residual(linear,[A,b],redshift,distance,distance_err)
plotHist(residuals)

<a name='exercises_5_1'></a>     

| [Top](#section_5_0) | [Restart Section](#section_5_1) | [Next Section](#section_5_2) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-5.1.1</span>

Consider the residual distribution found above, but now divide by the uncertainty of each measurement, that is, following the formula:

$$
\begin{equation}
y^{resid}_{i} = \frac{f(x_{i})-y_{true, i}}{\sigma_{y,i}}
\end{equation}
$$

If we plot this, and it is truly Gaussian, what are the *expected* values of the mean and stdev of the normalized residual distribution? Enter your answer as a list of two numbers with precision 1e-3: `[mean, stdev]`.

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-5.1.2</span>

Now use the data to compute the mean and standard deviation of the distribution defined by:

$$
\begin{equation}
y^{resid}_{i} = \frac{f(x_{i})-y_{true, i}}{\sigma_{y,i}}
\end{equation}
$$

**Hint: There is an option to do this in the function that we previously defined.**

How do the actual values compare to the values of an ideal Gaussian? Enter your answers found using the data as a list of two numbers with precision 1e-3: `[mean, stdev]`.


You may wish to use the starting code below.

In [ ]:
#>>>EXERCISE: L5.1.2
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

residuals=#YOUR CODE HERE
plotHist(residuals)

print("Ideally we should have a Gaussian with mean(residuals)=0 Instead we have",residuals.mean())
print("Ideally we should have a Gaussian with std(residuals)=1. Instead we have",residuals.std())


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-5.1.3</span>

Load the supernova data for entries up to a redshift of 0.4. You can do that by completing the line of code below. Now, compute the residuals dividing by the error, as above. What is the mean and RMS of the residuals now? Why are the residuals not a Gaussian shape? 

Enter your answer as a list of two numbers with precision 1e-3: `[mean, stdev]`.

In [ ]:
#>>>EXERCISE: L5.1.3
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

redshift02,distance02,distance02_err = load(label,0.4)

#Find these parameters by running the regression using regress()
A02,b02=#YOUR CODE HERE

plotAll(redshift02,distance02,distance02_err,A02,b02)

#plot regression to check
residuals02=#YOUR CODE HERE
plotHist(residuals02)

print("Ideally we should have a Gaussian with mean(residuals)=0 Instead we have",residuals.mean())
print("Ideally we should have a Gaussian with std(residuals)=1. Instead we have",residuals.std())


<a name='section_5_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L5.2 Extracting Uncertainty For Linear Fit</h2>  

| [Top](#section_5_0) | [Previous Section](#section_5_1) | [Exercises](#exercises_5_2) | [Next Section](#section_5_3) |


<a name='exercises_5_2'></a>     

| [Top](#section_5_0) | [Restart Section](#section_5_2) | [Next Section](#section_5_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 5.2.1</span>

Consider a fourth-order polynomial:

$$f(x) = a_{4}x^4 + a_{3}x^3 + a_{2}x^2 + a_{1}x + a_{0}$$  

If we are using this as our fit function for data consisting of `N` points, how many degrees of freedom do we have? Express your answer in terms of `N`.

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Exercise 5.2.2</span>

For the linear function in the example above, the variance in $A$ was expressed as: $\rm{Var}(A)=\frac{1}{N-2}\frac{\rm{Var}(y)}{\rm{Var}(x)}$.

What is the variance in $b$, expressed in terms of $\rm{Var}(y)$,  $\rm{Var}(x)$, $\bar{x}$, and $N$? Use the appropriate degrees of freedom, and express $\rm{Var}(y)$ as `Vary`, $\rm{Var}(x)$ as `Varx`, and $\bar{x}$ as `xbar`.

<a name='section_5_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L5.3 Computing Uncertainty</h2>  

| [Top](#section_5_0) | [Previous Section](#section_5_2) | [Exercises](#exercises_5_3) | [Next Section](#section_5_4) |


In [ ]:
#>>>RUN: L5.3-runcell01

residuals=residual(linear,[A,b],redshift,distance)
VarY=np.sum(residuals**2)/(len(redshift)-2)
VarA=VarY/variance(redshift)/(len(redshift)-2)
Varb=VarA*(redshift.mean())**2+VarY/(len(redshift)-2)
print("Hubbles Constant:",1e6*3e5/A,"+/-",1e6*3e5*math.sqrt(VarA)/A/A,"intercept",b,"+/-",math.sqrt(Varb))#Note 1e6 is from pc to Mpc and 3e5 is c in km/s


In [ ]:
#>>>RUN: L5.3-runcell02

weights=np.array([])
for pVal in distance_err:
    weights = np.append(weights,1./pVal/pVal)

#Now let's do it with weights
def variance_w(isamples,iweights):
    mean=np.average(isamples,weights=iweights)
    sumw=np.sum(iweights)
    tot=0
    for i0 in range(len(isamples)):
        tot+=iweights[i0]*(isamples[i0]-mean)**2
    return tot/sumw

def covariance_w(ixs,iys,iweights):
    meanx=np.average(ixs,weights=iweights)
    meany=np.average(iys,weights=iweights)
    sumw=np.sum(iweights)
    tot=0
    for i0 in range(len(ixs)):
        tot+=iweights[i0]*(ixs[i0]-meanx)*(iys[i0]-meany)
    return tot/sumw

def regress_w(redshift,weights,distance):
    varw=variance_w(redshift,weights)
    covw=covariance_w(redshift,distance,weights)
    Aw=covw/varw
    bw=np.average(distance,weights=weights)-Aw*np.average(redshift,weights=weights)
    return Aw,bw

Aw,bw=regress_w(redshift,weights,distance)
plotAll(redshift,distance,distance_err,Aw,bw)

def resid_w(func,args,distance,weights):
    residualsw=np.array([])
    for i0 in range(len(redshift)):
        pResid=linear(redshift[i0],args[0],args[1])-distance[i0]
        residualsw = np.append(residualsw,weights[i0]*pResid**2)
    return residualsw

residualsw = resid_w(linear,[Aw,bw],distance,weights)
sumw=np.sum(weights)
rsw=np.average(redshift,weights=weights)
sigmaw=np.sum(residualsw)/(len(redshift)-2)
VarAw=sigmaw*1./variance_w(redshift,weights)*1./sumw
Varbw=VarAw*(rsw)**2+sigmaw/sumw
    
print("Weighted Hubbles Constant:",1e6*3e5/Aw,"+/-",1e6*3e5*math.sqrt(VarAw)/Aw/Aw,"intercept",bw,"+/-",math.sqrt(Varbw))#Note 1e6 is from pc to Mpc and 3e5 is c in km/s
print()

#Now the previous
print("Unweighted Hubbles Constant:",1e6*3e5/A,"+/-",1e6*3e5*math.sqrt(VarA)/A/A,"intercept",b,"+/-",math.sqrt(Varb))#Note 1e6 is from pc to Mpc and 3e5 is c in km/s


<a name='exercises_5_3'></a>     

| [Top](#section_5_0) | [Restart Section](#section_5_3) | [Next Section](#section_5_4) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-5.3.1</span>

Why is the weighted Hubble's constant different from the unweighted one? Select the best answer below:

- Points with larger uncertainty pull on the fit-line more than points with smaller uncertainty.
- Points with smaller uncertainty pull on the fit-line more than points with larger uncertainty.
- It is impossible to know why the fit has changed.


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-5.3.2</span>

Compute the difference between the mean and the weighted mean to two significant digits; call this $\Delta_{\mathrm{Hubble}}$.

Also compute the uncertainty of the difference of the two measurements; call this $\sigma_{\mathrm{tot}}$. Assume they are separate, uncorrelated measurements, thus add the uncertainties in quadrature, i.e.:

$$\sigma^2_{\rm{tot}} = \sigma^2_{1} + \sigma^{2}_{2}$$

Enter your answer as a list of numbers with precision 1e-2: [$\Delta_{\mathrm{Hubble}}$, $\sigma_{\mathrm{tot}}$]


Are these variations consistent with each other? 

In [ ]:
#>>>EXERCISE: L5.3.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.


<a name='section_5_4'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L5.4 Introduction to Likelihood</h2>  

| [Top](#section_5_0) | [Previous Section](#section_5_3) | [Exercises](#exercises_5_4) | [Next Section](#section_5_5) |


In [ ]:
#>>>RUN: L5.4-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L05/slides_L05_04.html', width=970, height=550)

<a name='exercises_5_4'></a>     

| [Top](#section_5_0) | [Restart Section](#section_5_4) | [Next Section](#section_5_5) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-5.4.1</span>

What is the log likelihood evaluated at $p=40$%, given that you observe 56/100 people who claim they can smell asparagus after they pee?

You can use the starting code below, if you wish.

Report your answer with precision 1e-3.

In [ ]:
#>>>EXERCISE: L5.4.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def prob(p,nobs,ntrials):
    #nobs: the number of positive observations
    #ntrials: the total number of observations (trials)
    return #YOUR CODE HERE

print("Probability is ",prob(0.4,56,100))
print("Log Likelihood is", np.log(prob(0.4,56,100)))


>#### Follow-up 5.4.1a (ungraded)
>
>Plot the likelihood and log likelihood as a function of $p$, given the observation of a 56/100 result. Alternatively, plot the probability of attaining a given outcome vs. the number of positive outcomes (in other words, varying $x$, the number of positive results). How are these plots different?

<a name='section_5_5'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L5.5 An Example: Auger Data (Part 1)</h2>  

| [Top](#section_5_0) | [Previous Section](#section_5_4) | [Exercises](#exercises_5_5) | [Next Section](#section_5_6) |


In [ ]:
#>>>RUN: L5.5-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L05/slides_L05_05.html', width=970, height=550)

In [ ]:
#>>>RUN: L5.5-runcell01

import numpy as np
import csv
import math
import matplotlib.pyplot as plt


#Let's say we have
label='data/L05/events_a8_1space.dat'

def rad(iTheta):
    return iTheta/180. * math.pi

def rad1(iTheta):
    return iTheta/180. * math.pi-math.pi

def exposure(dec):
    theta_max = np.radians(60) # Maximum zenith angle in the dataset
    l = np.radians(-35.23) # Latitude of the center of the array (near Malargüe - Argentina)
    arg = (np.cos(theta_max) - np.sin(l)*np.sin(dec)) / (np.cos(l)*np.cos(dec))
    hm = np.arccos(arg.clip(-1, 1))
    return np.cos(l)*np.cos(dec)*np.sin(hm) + hm*np.sin(l)*np.sin(dec)

def load(label):
    dec=np.array([])
    ra=np.array([])
    az=np.array([])
    with open(label,'r') as csvfile:
        plots = csv.reader(csvfile,delimiter=' ')
        for pRow in plots:
            if '#' in pRow[0] or pRow[0]=='':
                continue
            dec = np.append(dec,rad(float(pRow[2])))
            ra  = np.append(ra,rad1(float(pRow[3])))
            az  = np.append(az,rad(float(pRow[4])))
    return dec,ra,az


dec,ra,az = load(label)

#Let's make a plot this is in local coordinates
color_map = plt.cm.Spectral_r
fig = plt.figure(figsize=(20, 8))
fig.add_subplot(111, projection='mollweide')
w=exposure(dec) #correct for the exposure rate at the latitude of the observatory.
image = plt.hexbin(ra, dec, cmap=color_map,gridsize=60, mincnt=1,C=w,reduce_C_function=np.sum)
plt.xlabel('R.A.')
plt.ylabel('Decl.')
plt.colorbar()
plt.grid(True)
plt.show()

#Now let's plot this in Galactic Coordinates
from astropy.coordinates import SkyCoord
from astropy import units
from astropy.coordinates import Galactic

coords = SkyCoord(ra=ra, dec=dec, unit='rad')
rap = coords.galactic.l.wrap_at(180 * units.deg).radian
decp = coords.galactic.b.radian

color_map = plt.cm.Spectral_r
fig = plt.figure(figsize=(20, 8))
fig.add_subplot(111, projection='mollweide')
image = plt.hexbin(rap, decp, cmap=color_map,gridsize=45, mincnt=1,reduce_C_function=np.sum)

plt.xlabel('R.A.')
plt.ylabel('Decl.')
plt.grid(True)
plt.colorbar(image, spacing='uniform', extend='max')
plt.show()


In [ ]:
#>>>RUN: L5.5-runcell02

from scipy import stats

def plotHist(iData,iNBins=30):
    #Ok enough of having fun, let's look at the asymmetry we observe in right asecion
    y0, bin_edges = np.histogram(iData, bins=iNBins)
    bin_centers = 0.5*(bin_edges[1:] + bin_edges[:-1])
    norm0=len(iData)*(bin_edges[-1]-bin_edges[0])/iNBins
    plt.errorbar(bin_centers,y0/norm0,yerr=y0**0.5/norm0,marker='.',drawstyle = 'steps-mid',linestyle='none')
    plt.xlabel("RA")
    plt.ylabel("Intensity")
    plt.show()
    return bin_centers,y0

print(len(ra))
_,_ = plotHist(ra)

<a name='exercises_5_5'></a>     

| [Top](#section_5_0) | [Restart Section](#section_5_5) | [Next Section](#section_5_6) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-5.5.1</span>

What is the range of declination (sky coordinates) available in the data (the data are given in units of radians).? Report your answer as a list of numbers, in units of radians, with precision 1e-3: `[dec_min, dec_max]`.

Hint: This can be found with a single line of code.


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-5.5.2</span>

Now let's look at the variations in the intensity as a function of RA for a subset of the data with a certain range of declination. Specifically, select the data with declination above the equator and plot it. Complete the code below to do this.

Do you see a trend similar to the one you see in $RA$? Select the best answer from the following:

- Yes, the trend is even more clear
- No, there is conclusively no trend
- There might be a slight trend, but the uncertainties are too large to tell


In [ ]:
#>>>EXERCISE: L5.5.2
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

ranew=#YOUR Code here

_,_ = plotHist(ranew)

>#### Follow-up 5.5.2a (ungraded)
>
>Systematically explore different slices of the data within the range `[dec_min, dec_max]`. Are there any trends that you notice?

<a name='section_5_6'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L5.6 An Example: Auger Data (Part 2)</h2>  

| [Top](#section_5_0) | [Previous Section](#section_5_5) | [Exercises](#exercises_5_6) | [Next Section](#section_5_7) |


In [ ]:
#>>>RUN: L5.6-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L05/slides_L05_06.html', width=970, height=550)

In [ ]:
#>>>RUN: L5.6-runcell01

#Let's say we have
NLeft=0
NRight=0
for i0 in range(len(ra)):
    if ra[i0] < 0:
        NLeft+=1
    else:
        NRight+=1
print("NLeft:",NLeft,"+/-",math.sqrt(NLeft),"NRight:",NRight,"+/-",math.sqrt(NRight),"total/2",len(ra)/2.)

plotHist(ra,2)

In [ ]:
#>>>RUN: L5.6-runcell02

def pvalues(NLeft,NRight):
    #Let's compute it
    lamb=(NLeft+NRight)/2. #average number of events
    pleft=stats.poisson.pmf(NLeft,lamb) #probability of left given averge
    pright=stats.poisson.pmf(NRight,lamb) #probability of right given averaged 
    pcheck=stats.poisson.pmf(int(lamb),lamb)#Most likely probability 
    print("Likelihood Ratio-left",pleft/pcheck,"Likelihood Ratio-right",pright/pcheck,"check",pcheck/pcheck)
pvalues(NLeft,NRight)

<a name='exercises_5_6'></a>     

| [Top](#section_5_0) | [Restart Section](#section_5_6) | [Next Section](#section_5_7) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-5.6.1</span>

Consider the analysis that we just performed, where we split the data into left and right RA. How many standard deviations is the count in the left bin, $N_{\mathrm{left}}$, from the expected (or average) count, $N_{\mathrm{avg}}$, assuming a uniform rate? Recall that $N_{\mathrm{avg}}=16093.5$ and $N_{\mathrm{left}} = 16600 \pm 128.84$.

Calculate the same number for the right bin, and report your answer as a list of two positive numbers `[num_stdev_left, num_stdev_right]`, with precision 1e-2.

In [ ]:
#>>>EXERCISE: L5.6.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

pass


In [ ]:
#>>>PROBLEM: L5.6.2

label_2='data/L05/events_a4_1space.dat'
dec_2,ra_2,az_2=load(label_2)

#compute asymmetry
NLeft_2=0
NRight_2=0

#YOUR CODE HERE

#PRINT THE COUNTS WITH UNCERTAINTY

#PRINT THE PROBABILITY (LIKELIHOOD RATIO)

#PLOT IT

<a name='section_5_7'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L5.7 Log-Likelihood and Chi-square</h2>  

| [Top](#section_5_0) | [Previous Section](#section_5_6) | [Exercises](#exercises_5_7) | [Next Section](#section_5_8) |


In [ ]:
#>>>RUN: L5.7-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L05/slides_L05_07.html', width=970, height=550)

In [ ]:
#>>>RUN: L5.7-runcell00

#For the purposes of this section, we will use the low energy data.
#Load it here, where we will redefine dec, ra, and az

label_2='data/L05/events_a4_1space.dat'
dec,ra,az=load(label_2)

In [ ]:
#>>>RUN: L5.7-runcell01

_,xis=plotHist(ra)

#Now let's define the log of a Poisson distribution (see above)
def logpoisson(lamb): #x is our lambda and y0 is our data
    lTot=0
    for xi in xis:
        test = math.factorial(xi)
        lTot = xi*np.log(lamb) - lamb  - math.log(test) + lTot
    return -1.*lTot

#Now lets take the mean of this distribution and compute labmda
lamb=xis.mean()
print("Log Likelihood",logpoisson(lamb),"Regular Likelihood",np.exp(logpoisson(lamb)))
x = np.linspace(lamb*0.75, lamb*1.55, 100)
plt.xlabel("$\lambda$")
plt.ylabel("$\log(\mathcal{L}(\lambda))$")
plt.plot(x, logpoisson(x));

#finally let's compute the minimum of this distribution
from scipy import optimize as opt
sol=opt.minimize_scalar(logpoisson, method='Brent')
print("minimum found:",sol,"Mean:",lamb.mean())

<a name='section_5_8'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L5.8 Minimizing</h2>  

| [Top](#section_5_0) | [Previous Section](#section_5_7) | [Exercises](#exercises_5_8) | [Next Section](#section_5_9) |


In [ ]:
#>>>RUN: L5.8-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L05/slides_L05_08.html', width=970, height=550)

In [ ]:
#>>>RUN: L5.8-runcell01

#Log likelihood of a gaussian distribution of our data from above
def loggaus(lamb):
    lTot=0
    for xi in xis:
        lTot = lTot+(0.5/(lamb+1e-5))*(xi-lamb)**2
        lTot = lTot-0.5*np.log(math.pi*2*lamb)
    return lTot

#chi2 distribution of our data from above.  
def chi2(lamb):
    lTot=0
    for xi in xis:
        lTot = lTot+(1./(lamb+1e-10))*(xi-lamb)**2
    return lTot


lamb=xis.mean()
print("Gaussian Likelihood at minimum",loggaus(lamb),lamb,len(xis))
print("chi2 value at minium",chi2(lamb))

x = np.linspace(lamb*0.75, lamb*1.55, 100)
plt.plot(x, loggaus(x),label='gaus');
plt.plot(x, chi2(x)/2.,label='chi2/2');
plt.xlabel("$\lambda$")
plt.ylabel("$\log(\mathcal{L}(\lambda))$")
plt.legend(loc='lower right')
plt.show()

#Now let's minimize the two
from scipy import optimize as opt
sol1=opt.minimize_scalar(loggaus, method='Brent')
print("Gaussian Minimum",sol1.x)
sol2=opt.minimize_scalar(chi2, method='Brent')
print("Chi2 Minimum",sol2.x)

In [ ]:
#>>>RUN: L5.8-runcell02

#Now let's look at our chi2 distribution and see how this compares
x = np.linspace(0,80)
chi2d=stats.chi2.pdf(x,30) # w0 bins
plt.plot(x,chi2d,label='chi2')
plt.axvline(chi2(lamb), c='red')
plt.legend(loc='lower right')
plt.xlabel("$\chi^{2}(x)$")
plt.ylabel("p-value")
plt.show()

In [ ]:
#>>>RUN: L5.8-runcell03

def chi2min(ilamb):
    minchi2=chi2(xis.mean())+1 #This is the minmum over the data
    return chi2(ilamb)-minchi2

lamb=xis.mean()
sol1=opt.root_scalar(chi2min,bracket=[lamb, lamb*1.1],method='brentq')
sol2=opt.root_scalar(chi2min,bracket=[lamb*0.9, lamb],method='brentq')
print(sol1)
print(sol2)
print("sol1",xis.mean()+math.sqrt(xis.mean()/len(xis)))
print("sol2",xis.mean()-math.sqrt(xis.mean()/len(xis)))


minlog=chi2(lamb)
x = np.linspace(lamb*0.995, lamb*1.005, 50)
plt.plot(x, chi2(x),label='chi2');
plt.axvline(sol1.root, c='red',label="$\hat{\lambda}\pm1\sigma$")
plt.axvline(lamb, c='blue',label="$\hat{\lambda}$")
plt.axvline(sol2.root, c='red')
plt.legend(loc='lower right')
plt.xlabel("$\lambda$")
plt.ylabel("$\log(\mathcal{L}(\lambda))$")
plt.show()

<a name='exercises_5_8'></a>     

| [Top](#section_5_0) | [Restart Section](#section_5_8) | [Next Section](#section_5_9) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-5.8.1</span>

Now if we treat $\chi^2$ as that of a single variable with a variance of $1$ (i.e., for some $\lambda$), we can write

$$
\chi^{2} = \left(\frac{\lambda -\hat{\lambda}}{\sigma_{\lambda}}\right)^{2}
$$

where $\hat{\lambda}$ is the best fit and $\sigma_{\lambda}$ is the uncertainty in $\lambda$.

What is the $\Delta \chi^2$ value from the minimum $\chi^{2}$ corresponding to two standard deviations in $\lambda$ from the best fit $\hat{\lambda}$? Enter an integer number for your answer.

<a name='section_5_9'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L5.9 Comparison Using lmfit</h2>     

| [Top](#section_5_0) | [Previous Section](#section_5_8) | [Exercises](#exercises_5_9) |


In [ ]:
#>>>RUN: L5.9-runcell01

#solution
label8='data/L05/events_a8_1space.dat'
dec,ra,az=load(label8)
nbins=30
_,xis=plotHist(ra,nbins)

#compute the mean over the bins
lamb=xis.mean()

#minimize it
sol2=opt.minimize_scalar(chi2, method='Brent')
print("chi2 minimized value",lamb.mean())

#Now let's look at our chi2 distribution and see how this compares
x = np.linspace(0,80)
chi2d=stats.chi2.pdf(x,nbins) # 30 bins
plt.plot(x,chi2d,label='chi2')
plt.axvline(chi2(lamb), c='red')
plt.legend(loc='lower right')
plt.show()

#now let's plot the mimum and uncertainty
sol0=opt.minimize_scalar(chi2, method='Brent')
lamb=sol0.x
sol1=opt.root_scalar(chi2min,bracket=[lamb, lamb*1.01],method='brentq')
sol2=opt.root_scalar(chi2min,bracket=[lamb*0.99, lamb],method='brentq')
print(sol0)
print(sol1)
print(sol2)
print("sol1",xis.mean()+math.sqrt(xis.mean()/len(xis)))
print("sol2",xis.mean()-math.sqrt(xis.mean()/len(xis)))

minlog=chi2(lamb)
x = np.linspace(lamb*0.99, lamb*1.01, 50)
plt.plot(x, chi2(x),label='chi2');
plt.axvline(sol1.root, c='red',label="$\hat{\lambda}\pm1\sigma$")
plt.axvline(lamb, c='blue',label="$\hat{\lambda}$")
plt.axvline(sol2.root, c='red')
plt.legend(loc='lower right')
plt.xlabel("$\lambda$")
plt.ylabel("$\log(\mathcal{L}(\lambda))$")
#plt.legend(loc='lower right')
plt.show()

In [ ]:
#>>>RUN: L5.9-runcell02

import lmfit
#Plot a constant function
def f(x,a):
    return a

def prephist(iRA):
    y0, bin_edges = np.histogram(iRA, bins=30)
    x0 = 0.5*(bin_edges[1:] + bin_edges[:-1])
    y0 = y0.astype('float')
    return x0,y0,1./(y0**0.5)

def plothist(iRA):
    x,y,xweights=prephist(iRA)
    model  = lmfit.Model(f)
    p = model.make_params(a=1000)
    result = model.fit(data=y,x=x, params=p, weights=xweights)
    lmfit.report_fit(result)
    print(result.params.items(),result.params["a"].value)
    plt.errorbar(x,y,yerr=y**0.5,c='black',marker='.',linestyle = 'None')
    x = np.linspace(x[0],x[-1], 50)
    y=np.array([])
    for pX in x:
        pOut=f(pX,result.params["a"].value)
        y=np.append(y,pOut)
    plt.plot(x,y)
    plt.xlabel("RA")
    plt.ylabel("$N_{events}$")
    plt.show()
    
label8='data/L05/events_a8_1space.dat'
dec,ra8,az=load(label8)
plothist(ra8)

label4='data/L05/events_a4_1space.dat'
dec,ra4,az=load(label4)
plothist(ra4)


In [ ]:
#>>>RUN: L5.9-runcell03

#Answer: 
print("Answer:",stats.norm.cdf(0,1)*100,"% Above the line")
print("Answer:",stats.norm.cdf(0,1)*100,"% Below the line")

In [ ]:
#>>>RUN: L5.9-runcell04

def fnew(x,a,b):
    pVal=b*np.sin(x)
    return a+pVal

label8='data/L05/events_a8_1space.dat'
dec,ra8,az=load(label8)
x,y,xweights=prephist(ra8)
print(x,y)

model  = lmfit.Model(fnew)
p = model.make_params(a=1000,b=10)
result = model.fit(data=y,x=x, params=p, weights=xweights)
lmfit.report_fit(result)
plt.figure()
result.plot()
plt.show()

In [ ]:
#>>>RUN: L5.9-runcell05

#Now let's look at our chi2 distribution and see how this compares
x = np.linspace(0,80)
chi2d=stats.chi2.pdf(x,40) # 40 bins
plt.plot(x,chi2d,label='chi2')
plt.axvline(result.chisqr, c='red')
plt.legend(loc='lower right')
plt.show()

<a name='exercises_5_9'></a>   

| [Top](#section_5_0) | [Restart Section](#section_5_9) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-5.9.1</span>

Repeat the preceding fit using the function: 

$$
\begin{equation}
 f(x) = a x + b 
\end{equation}
$$

What is the $\chi^{2}$ value? Enter a number with precision 1e-3.

Given these results, which fit is better? 

In [ ]:
#>>>EXERCISE: L5.9.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.
